In [1]:
import pandas as pd
from bs4 import BeautifulSoup #library for pulling data out of HTML and XML files
import requests

print('Libraries imported and ready!')

Libraries imported and ready!


Let's scrap the Wikipedia Table

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, "lxml")

table = soup.find('table')

rowData = []

for tr in table.find_all('tr'): 
    td = tr.find_all("td")
    row = [tr.text.strip() for tr in td] #.strip() to remove all the new line spetial character \n 
    
    # Process the info that have an assigned borough and ignore the cells that has Not Assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        rowData.append(row)

df_Toronto = pd.DataFrame(rowData, columns = ["PostalCode", "Borough", "Neighborhood"])
df_Toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df_Toronto.shape

(103, 3)

Get the Latitude and Longitude of each neighborhood in the city of Toronto

In [4]:
url = 'http://cocl.us/Geospatial_data'
df_geoData=pd.read_csv(url)
df_geoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df_geoData.shape

(103, 3)

We have two tables of the same shape. Now we can join both tables.

In [6]:
df_geoData.rename(columns = {'Postal Code' : 'PostalCode'}, inplace = True)
df_Toronto.set_index('PostalCode')
df_geoData.set_index('PostalCode')
toronto_postalCode = pd.merge(df_Toronto, df_geoData)
toronto_postalCode.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
